### Трюк с оптимизацией объема (размера) памяти потребляемой моделями глубокого обучения:
----
#### Квантизация (Quantization)
Как мы с вами поняли из опыта работы с нейросетями в PyTorch, нейросеть - это просто реализация вычислительного графа. По умолчанию наши устройства и процессор настроен на работу с 32-х битной системой адресов (у некоторых 64-х битная). То есть по умолчанию размер памяти, который резервируется под число с плавающей точкой - 32 / 64 бита. В случае обучения очень глубоких моделек - это очень много и выливается в проблемы хранения большого объема весов и соответственно загрузке и использованию обученной модели где-то в дальнейшем на сервере (чем больше объем, тем труднее оптимально развернуть модельку в качестве сервиса).

Работа квантизации происходит при помощи различных манипуляций:
1. Функция маппинга (mapping function) - для того, чтобы связать float и int и преобразовать данные весов. Обычно это простая линейная трансформация вида Q(r) = round(r/S+Z) где r - вход а S и Z параметры квантизации. Обратная функция r_r = (Q(r) - Z) * S

S еще называют параметром шкалирования (scaling factor) Z - околонулевая опора (нулевая опора, zero-point).

2. Калибровки (где рассчитываются параметры шкалирования)
3. Сама квантизация.
----
Что предлагает PyTorch:
----
У нас существует возможность понизить точность (precision) для числа с плавающей точкой, что позволит освободить нам часть памяти при расчетах и соответственно получить +/- похожий инференс модели при ее тренировке и использовании.
Квантизация - это техники для расчета и доступа к памяти с данными пониженной точности (в случае машинного обучения). Техник квантизации очень много, они позволяют уменьшать размер модели, сокращать объем потребляемой памяти и увеличивать инференс модели при ее использовании в продакшене.

Не будем углубляться в низкоуровневую реализацию процесса квантизации, кому интересно как это работает под капотом и какие процессы запускаются во время работы, в конце ноутбука - ссылки на обоснование квантизации и эмпирическое доказательство работы процесса.

**Нам важно как это использовать на практике и как это может нам помочь.**

Самый простой и быстрый метод квантизации - понизить уровень расчетов до precision int8. Посмотрим как это реализуется в PyTorch:

In [1]:
import torch
import warnings
warnings.filterwarnings("ignore")

class VanillaLeNet5(torch.nn.Module):
    def __init__(self):
        super(VanillaLeNet5, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = torch.nn.functional.max_pool2d(
            torch.nn.functional.relu(self.conv1(x)), (2, 2))
        x = torch.nn.functional.max_pool2d(
            torch.nn.functional.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement()/x.shape[0]))
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [2]:
# Создадим экземпляр модели, архитектуру которой мы построили выше
model = VanillaLeNet5()

#### Как мы уже знаем, все операции реализованы при помощи использования float32, мы можем убедиться в этом дополнительно если пройдемся по каждому слою модели и ее параметрам:

In [ ]:
for layer, datatype in model.named_parameters():
    print("Layer: {}\nDataType: {}\n".format(layer, datatype.dtype))

#### Один из подходов квантизации -  это просто сменить тип данных до более низкого пресижена. Это делается при помощи метода <model_instance>.half()

In [ ]:
model = model.half()
for layer, datatype in model.named_parameters():
    print("Layer: {}\nDataType: {}\n".format(layer, datatype.dtype))

#### Очень часто может быть такое, что у нас не стоит задача провести квантизацию на всем участке графа нейронной сетки, а просто уменьшить количество потребляемой памяти отдельного участка графа (например самого прожорливого). Для такого кейса в PyTorch реализованы три механизма квантизации графа:

----
1. Динамическая квантизация (Dynamic Quantization).
2. Статическая квантизация после обучения (Post-Training Static Quantization).
3. Обучение c квантизацией (Quantization-Aware Training).
----
Рассмотрим каждое из них по отдельности:

#### 1. Динамическая Квантизация (Dynamic Quantization):
----
Динамическая квантизация используется когда объем вычислений для весовых матриц (то сколько мы должны потратить ресурсов памяти для обучения) ограничен нашим железом и нашей пропускной способностью. То есть в какой-то момент времени мы упремся в нехватку памяти и возможна ошибка наподобие MemoryError либо MemorySegmentationError. Это частый случай при работе с такими архитектурами как **LSTM, RNN, GRU, Bidirectional Encoder Representation, Attention Mechanisms, Encoder-Decoder, большие Embedding, Transformers (наподобие BERT, CLIP и т.д.).**

Динамическая квантизация один из легких вариантов. Что делает динамическая квантизация в PyTorch - меняет precision для функций активаций на лету под int8 для указанного (указанных) слоев. Но этап сохранения и записи все равно происходит в float типе.

----


**!FYI**:
Динамическая квантизация зависит от типа архитектуры процессора на которой вы ее проводите. На данный момент в PyTorch поддерживаются **только 2 из них - x86 (fbgemm) и ARM (qnnpack).**


----

Рассмотрим небольшой пример:

In [8]:
import torch

dynamically_quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear}, # Указываем необходимые слои нейросети на которых необходимо провести квантизацию
    dtype=torch.qint8
)

#### 2. Статическая Квантизация после обучения (Post-Training Static Quantization):
----

Используется чтобы еще больше снизить "прожорливость" модели путем манипуляций с распределениями различных весов в нейросети и полученных значений после активаций. Этот тип квантизации позволяет нам передавать квантизированные значения между операциями без преобразования между числами с плавающей запятой и целыми числами в памяти.

Статическая квантизация после обучения требует чтобы мы сами настроили параметры. После того как будут откалиброваны параметры модельки ее можно будет конверитировать в квантизированную версию:

In [9]:
static_quantized_model = VanillaLeNet5()

static_quantized_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(static_quantized_model, inplace=True)
torch.quantization.convert(static_quantized_model, inplace=True)

VanillaLeNet5(
  (conv1): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (fc1): QuantizedLinear(in_features=400, out_features=120, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc2): QuantizedLinear(in_features=120, out_features=84, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc3): QuantizedLinear(in_features=84, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
)

#### 3. Обучение к Квантизацией (Quantization-Aware training):
----

Обычно этот вариант не снижает инференс модели. В данном варианте все веса и активации "псевдо квантизируются" во время обучения (Forward Propagation / Backward Propagation). Float тип данных конвертируется в int тип во время вычислений:

In [12]:
quantazed_aware_training_model = VanillaLeNet5()
quantazed_aware_training_model.qconfig = torch.quantization.get_default_qat_qconfig("fbgemm")

torch.quantization.prepare(quantazed_aware_training_model, inplace=True)
torch.quantization.convert(quantazed_aware_training_model, inplace=True)

VanillaLeNet5(
  (conv1): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (fc1): QuantizedLinear(in_features=400, out_features=120, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc2): QuantizedLinear(in_features=120, out_features=84, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc3): QuantizedLinear(in_features=84, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
)

#### Рассмотрим на примере использования архитектуры ResNet18 и датасета MNIST:

In [24]:
import os
import time
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import warnings
warnings.filterwarnings("ignore")

#### Загрузка и формирование датасета MNIST Для тренировки:

In [17]:
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(root='./../data',
                                                                      train=True,
                                                                      download=True,
                                                                      transform=torchvision.transforms.Compose([
                                                                          torchvision.transforms.ToTensor(),
                                                                          torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                                                      ])),
                                           batch_size = 64,
                                           shuffle=True,
                                           pin_memory=True) # параметр для более оптимального переноса между устройствами GPU/CPU <-> CPU/GPU

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(root='./../data',
                                                                      train=False,
                                                                      download=True,
                                                                      transform=torchvision.transforms.Compose([
                                                                          torchvision.transforms.ToTensor(),
                                                                          torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                                                      ])),
                                           batch_size = 64,
                                           shuffle=True,
                                           pin_memory=True) # параметр для более оптимального переноса между устройствами GPU/CPU <-> CPU/GPU

In [ ]:
images, labels = next(iter(train_loader))

print(images.shape)
print(labels.shape)

figure = plt.figure()
num_of_images = 20
for index in range(1, num_of_images + 1):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')

#### Возьмем и реализуем ResNet18 модельку, архитектура взята сразу с https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

In [25]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None, quantize=False):
        super(BasicBlock, self).__init__()
        self.quantize = quantize
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        # FloatFunction()
        self.skip_add = nn.quantized.FloatFunctional()


    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # Notice the addition operation in both scenarios
        if self.quantize:
            out = self.skip_add.add(out, identity)
        else:
            out += identity

        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block=BasicBlock, layers=[2, 2, 2, 2], num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None, mnist=False, quantize=False):
        super(ResNet, self).__init__()
        self.quantize = quantize
        if mnist:
            num_channels = 1
        else:
            num_channels = 3
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead.
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(num_channels, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer, quantize=self.quantize))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer, quantize=self.quantize))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # Input are quantized
        if self.quantize:
            x = self.quant(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        # Outputs are dequantized
        if self.quantize:
            x = self.dequant(x)

        return x

    def forward(self, x):
         # See note [TorchScript super()]
        return self._forward_impl(x)

#### Проведем тренировку:

In [26]:
def train(args, model, device, train_loader, optimizer, epoch):
    """ Train the model with given dataset

    Args:
        args: args like log interval
        model: ResNet model to train
        device: CPU/GPU
        train_loader: dataset iterator
        optimizer: optimizer to update weights
        epoch: number of epochs to train for
    """
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(F.log_softmax(output, dim=-1), target)
        loss.backward()
        optimizer.step()

        if batch_idx % args["log_interval"] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [27]:
def main():

    batch_size = 64
    epochs = 5
    lr = 0.01
    momentum = 0.5
    seed = 1
    log_interval = 500
    save_model = True
    no_cuda = False

    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(seed)
    device = torch.device("cuda" if use_cuda else "cpu")
    model = ResNet(num_classes=10, mnist=True).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    args = {}
    args["log_interval"] = log_interval
    for epoch in range(1, epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)

    if (save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")

main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.695870
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.238433
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.030870
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.054056
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.009927
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.013909
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.013416
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.004008
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.005189
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.005069


#### Тестирование модели без квантизации и с квантизацией:

In [28]:
def print_size_of_model(model):
    """ Print the size of the model.
    Args:
        model: model whose size needs to be determined
    """
    torch.save(model.state_dict(), "temp.p")
    print('Size of the model(MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [29]:
def test(model, device, test_loader, quantize=False, fbgemm=False):
    model.to(device)
    model.eval()

    # Testing with qauntization if quantize=True
    if quantize:
        modules_to_fuse = [['conv1', 'bn1'],
                   ['layer1.0.conv1', 'layer1.0.bn1'],
                   ['layer1.0.conv2', 'layer1.0.bn2'],
                   ['layer1.1.conv1', 'layer1.1.bn1'],
                   ['layer1.1.conv2', 'layer1.1.bn2'],
                   ['layer2.0.conv1', 'layer2.0.bn1'],
                   ['layer2.0.conv2', 'layer2.0.bn2'],
                   ['layer2.0.downsample.0', 'layer2.0.downsample.1'],
                   ['layer2.1.conv1', 'layer2.1.bn1'],
                   ['layer2.1.conv2', 'layer2.1.bn2'],
                   ['layer3.0.conv1', 'layer3.0.bn1'],
                   ['layer3.0.conv2', 'layer3.0.bn2'],
                   ['layer3.0.downsample.0', 'layer3.0.downsample.1'],
                   ['layer3.1.conv1', 'layer3.1.bn1'],
                   ['layer3.1.conv2', 'layer3.1.bn2'],
                   ['layer4.0.conv1', 'layer4.0.bn1'],
                   ['layer4.0.conv2', 'layer4.0.bn2'],
                   ['layer4.0.downsample.0', 'layer4.0.downsample.1'],
                   ['layer4.1.conv1', 'layer4.1.bn1'],
                   ['layer4.1.conv2', 'layer4.1.bn2']]
        model = torch.quantization.fuse_modules(model, modules_to_fuse)
        if fbgemm:
            model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
        else:
            model.qconfig = torch.quantization.default_qconfig
        torch.quantization.prepare(model, inplace=True)
        model.eval()
        with torch.no_grad():
            for data, target in train_loader:
                model(data)
        torch.quantization.convert(model, inplace=True)

    print(model)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            st = time.time()
            output = model(data)
            et = time.time()
            test_loss += F.nll_loss(F.log_softmax(output, dim=-1), target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print("========================================= PERFORMANCE =============================================")
    print_size_of_model(model)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print('Elapsed time = {:0.4f} milliseconds'.format((et - st) * 1000))
    print("====================================================================================================")

### Baseline Inference - моделька без квантизации:

In [30]:
device = 'cpu'
encoder = ResNet(num_classes=10, mnist=True)
loaded_dict_enc = torch.load('mnist_cnn.pt', map_location=device)
encoder.load_state_dict(loaded_dict_enc)
test(model=encoder, device=device, test_loader=test_loader)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05,

### Quantized Inference - моделька с квантизацией:

In [31]:
device = 'cpu'
encoder = ResNet(num_classes=10, mnist=True, quantize=True)
loaded_dict_enc = torch.load('mnist_cnn.pt', map_location=device)
encoder.load_state_dict(loaded_dict_enc)
test(model=encoder, device=device, test_loader=test_loader, quantize=True)

ResNet(
  (conv1): QuantizedConv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.1244458258152008, zero_point=61, padding=(3, 3))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.09917448461055756, zero_point=66, padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.10016588121652603, zero_point=60, padding=(1, 1))
      (bn2): Identity()
      (skip_add): QFunctional(
        scale=0.13375742733478546, zero_point=45
        (activation_post_process): Identity()
      )
    )
    (1): BasicBlock(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.10260023176670074, zero_point=61, padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inp

### Вывод:

| Модель    | Размер Mb | Точность / Время инверенса |
|-----------|:---------:|---------------------------:|
| Baseline  |   44 Mb   |                99% / 23 ms |
| Quantized |   11 Mb   |                 99% / 7 ms |


#### Дополнительная информация:
1. https://pytorch.org/docs/stable/quantization.html#general-quantization-flow
2. https://pytorch.org/blog/quantization-in-practice/